In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests
def get_page_html():
    url = 'https://www.costco.com/kirkland-signature-coffee-organic-pacific-bold-k-cup-pod%2c-120-count.product.4000147442.html'
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}
    page = requests.get(url, headers=headers)
    return page.content
info = get_page_html()

In [ ]:
print(info)

In [4]:
## check if disc active
from bs4 import BeautifulSoup
soup = BeautifulSoup(info, 'html.parser')


In [5]:
disc_active_divs = soup.find_all("div", {"class": "marketing-container form-group"})


In [ ]:
disc_active_divs

In [ ]:
"https://www.costco.com/s?keyword=OFF&dept=All&sortBy=item_page_views%2Bdesc&refine="

In [20]:
import requests
def get_page_html():
    url = "https://www.costco.com/grocery-household.html?keyword=OFF&dept=All&sortBy=item_page_views+desc"
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}
    page = requests.get(url, headers=headers)
    return page.content
info = get_page_html()

In [ ]:
info

In [ ]:
## check if disc active
from bs4 import BeautifulSoup
soup = BeautifulSoup(info, 'html.parser')
soup

In [23]:
disc_active_divs = soup.find_all("div", {"class": "product-tile-set"})


In [ ]:
disc_active_divs[0].find_all("a", {"class": "product-image-url"})[0].get("href")

In [73]:
import requests
def get_page_html():
    url = "https://www.costco.com/charmin-ultra-soft-bath-tissue%2c-2-ply%2c-213-sheets%2c-30-rolls.product.4000221238.html"
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"}
    page = requests.get(url, headers=headers)
    return page.content
info = get_page_html()

In [ ]:
## check if disc active
from bs4 import BeautifulSoup
soup = BeautifulSoup(info, 'html.parser')
disc_active_divs = soup.find_all("div", {"class": "marketing-container form-group"})
disc_active_divs

In [ ]:
soup

In [ ]:
scripts = soup.find_all("script")
for script in scripts:
    if script.string and 'var products' in script.string:
        js = script.string

print(js)

In [ ]:
"""var products = [

	[

		{
		
			"shipToWarehouse": false,
		
			"isLocInvItem":"true",
			"pricePerUnit" : "",
			"unitOfMeasure" : "Price Per 100 SQ FT:",
			"unpriced" : false,
			"single":true,
			"catentry" : "3074457345620209517",
			"partNumber" : "1773119",
			"programType" : "3rdPartyDelivery,2DayDelivery,GoogleGrocery,LocationControlledInventory,InWarehouse,WarehouseDelivery",
			"manufacturerPartNumber" : " ",
			"defaultShipModeId" : "10654",
			"vendorModelNumber" : "",
			"price" : "",
			"productName" : "Charmin Ultra Soft Bath Tissue, 2-Ply, 213 Sheets, 30 Rolls",
			"productUrl" : "https://www.costco.com/charmin-ultra-soft-bath-tissue%2c-2-ply%2c-213-sheets%2c-30-rolls.product.1773119.html",
			"listPrice" : "LTEuMA==",
			"date" : "",
			"date2" : "",
			"type" : "0",
			"message" : "0",
			"zipcodeAvailabilityMessage" : "0",
			"zipcodeAvailabilityResponseCode" : "success",
			"inventory" : "IN_STOCK",
			"minQty" : "1",
			"maxQty" : "9999",
			"ordinal" : "11551.0",
			"itemSequence" : "",
			"img_url" : "https://bfasset.costco-static.com/U447IH35/as/2q7sfmpkk4bjmm66sjh8/1773119-847__1?auto=webp&amp;format=jpg&amp;width=300&amp;height=300&amp;fit=bounds&amp;canvas=300,300",
			"plainPackaging" : "2",
			"eddVendorName" : "",
			"liveEDD" : "false",
			"options" : [],
			"itemLimitOneQty" : "false",
			"isDependentProductConfigured": "false",
			"feeRegions" : [
				
  			],
  			"feeOptions" : [
  				
  			],
  			"regionPrices" : [
  				
  			]
		}
	
	]
];
"""
import re
pattern = r"var products\s*=\s*\[[\s\S]*?\];"
match = re.search(pattern, js)
match.group(0)

In [68]:
### turn the above into a json
import json
products = match.group(0).split("= ")[1].replace(";", "")
products = json.loads(products)

In [ ]:
products[0][0]["productName"]


In [6]:
from enum import Enum
import structlog
from bs4 import BeautifulSoup
from pydantic import BaseModel
import httpx
import time
import random
import re
import json

def jitter(max_time: int = 5):
    time.sleep(random.randint(0, max_time))

logger = structlog.get_logger()


class Catagories(str, Enum):
    COFFEE = "coffee"
    MEAT = "meat"
    PAPER_GOODS = "paper_goods"
    CLEANING_SUPPLIES = "cleaning_supplies"
    PERSONAL_CARE = "personal_care"
    SNACKS = "snacks"
    BEVERAGES = "beverages"



class CostcoCategories(str, Enum):
    GROCERY = 'grocery-household'

CLIENT = httpx.Client(
    headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36"},
    verify=False,
    timeout=30,
)



def get_all_products(catagories: CostcoCategories, page_size: int = 24, page_max: int = 20) -> list[str]:
    page_data = []
    for page in range(1, page_max):
        try:
            url = f"https://www.costco.com/{catagories.value}.html?currentPage={page}&dept=All&pageSize={page_size}&sortBy=item_page_views+desc&keyword=OFF&dept=All"
            page = CLIENT.get(url)
            ## wait a bit
            jitter()
            page_data.append(
                BeautifulSoup(page.content, 'html.parser')
            )
            if page == 20:
                break
        except Exception as e:
            logger.error(f"Failed to get page {page} with error {e}")
            break
    ## flatten the list
    return page_data
    

def get_all_product_info(category_data: list[str]) -> list[str]:
    product_info = []
    for product in category_data:
        try:
            url = product.find_all("a", {"class": "product-image-url"})
            for uri in url:
                href = uri.get("href")
                page = CLIENT.get(href)
                product_info.append(
                    BeautifulSoup(page.content, 'html.parser')
                )
                jitter()
        except Exception as e:
            logger.error(f"Failed to get product info with error {e}")
    return product_info

def get_all_names_and_discounts(product_info: list[str]) -> list[Product]:
    products = []
    for product in product_info:
        scripts = product.find_all("script")
        discounts = product.find_all("div", {"class": "marketing-container form-group"})
        for script in scripts:
            if script.string and 'var products' in script.string:
                js = script.string
                pattern = r"var products\s*=\s*\[[\s\S]*?\];"
                match = re.search(pattern, js)
                item = Product(
                    name=json.loads(match.group(0).split("= ")[1].replace(";", ""))[0][0]["productName"],
                    discount=discounts[0].text
                )
                products.append(
                    item
                )
    return products



def run_category(category: CostcoCategories) -> list[Product]:
    products = get_all_products(category)
    product_info = get_all_product_info(products)
    return get_all_names_and_discounts(product_info)







In [ ]:
products = get_all_products(category)
product_info = get_all_product_info(products)


In [7]:
grocery = get_all_products(CostcoCategories.GROCERY, page_max=2)

In [8]:
product_info = get_all_product_info(grocery)


In [103]:
product_info[0]


<!DOCTYPE html>

<html lang="en-us">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<script type="text/javascript">
		window.assetDomain = 'https://my.costco.com/assets/'; // Need to get from WCS property
	</script>
<script>
				var isJQueryUpgraded = true;
			</script>
<!-- START SeoMetaData.jsp -->
<meta content="en-us" http-equiv="content-language">
<meta content="3.0.87076.0" name="currentBuildNumber">
<meta content="3.0.33624.0" name="currentSolrBuildNumber">
<meta content="www.costco.com " name="ServerName"/>
<meta content="xxx.xxx.xxx.44" name="LocalAddress"/>
<meta automation-id="metadataLocalName" content="TP222" name="LocalName"/>
<meta content="" name="google-site-verification"/>
<link href="/wcsstore/CostcoGLOBALSAS/images/favicon.ico" rel="icon" type="image/x-icon"/>
<link href="/wcsstore/CostcoGLOBALSAS/images/favicon.ico" rel="shortcut icon" type="image/x-icon"/>

In [125]:

'''<link href="https://www.costco.com/kirkland-signature-coffee-organic-breakfast-blend-k-cup-pod%2c-120-count.product.4000146862.html" rel="canonical">'''
canonical_link = product_info[0].find("link", {"rel": "canonical"})
canonical_link['href'] if canonical_link else None


'https://www.costco.com/kirkland-signature-coffee-organic-breakfast-blend-k-cup-pod%2c-120-count.product.4000146862.html'

TypeError: 'NoneType' object is not subscriptable

In [ ]:
import re
from smart_shopper.models import Product

"""<script type="text/javascript">
 		            window.digitalData = {
 		                siteLang :  'en',
 		                siteReleaseEnv : 'prod',
 		                siteStore : 'bc',
 		                siteCountry : 'us',
 		                pageName : 'Product Detail | 4000278026',
 		                pageType : 'Product Detail',
 		                pageCrumbs : ["Grocery & Household Essentials", "Cleaning Supplies", "Laundry Detergent & Supplies"],
 		                product : {
 		                  pid : '4000278026',
 		                  sku : '1840548',
 		                  prodDetailType : 'na',
 		                  name : 'Downy Soft Liquid Fabric Softener, April Fresh, 257 Loads, 150 fl oz',
 		                  inventoryStatus : 'in stock',
 		                  priceMin : '13.99',
 		                  priceMax : '16.99',
 		                  membershipReq : 'na',  
 		                },
 		            }
 		            window.digitalDataEvents = [];
 		    </script>,"""



"""
window.digitalData = {
 		                siteLang :  'en',
 		                siteReleaseEnv : 'prod',
 		                siteStore : 'bc',
 		                siteCountry : 'us',
 		                pageName : 'Product Detail | 4000278026',
 		                pageType : 'Product Detail',
 		                pageCrumbs : ["Grocery & Household Essentials", "Cleaning Supplies", "Laundry Detergent & Supplies"],
 		                product : {
 		                  pid : '4000278026',
 		                  sku : '1840548',
 		                  prodDetailType : 'na',
 		                  name : 'Downy Soft Liquid Fabric Softener, April Fresh, 257 Loads, 150 fl oz',
 		                  inventoryStatus : 'in stock',
 		                  priceMin : '13.99',
 		                  priceMax : '16.99',
 		                  membershipReq : 'na',  
 		                },
 		            }

"""
canonical_link = product_info[0].find("link", {"rel": "canonical"})
canonical_link['href'] if canonical_link else None
pattern = r"window\.digitalData\s*=\s*\{[\s\S]*?\};?"
products = []
for script in scripts:
	if script.string:
		match = re.search(pattern, script.string)
		if match:
			name = re.search(r"name\s*:\s*'(.*)'", script.string).group(1)
			price_min = re.search(r"priceMin\s*:\s*'(.*)'", script.string).group(1)
			price_max = re.search(r"priceMax\s*:\s*'(.*)'", script.string).group(1)
			inventory_status = re.search(r"inventoryStatus\s*:\s*'(.*)'", script.string).group(1)
			page_crumbs = re.search(r"pageCrumbs\s*:\s*\[(.*)\]", script.string).group(1)
			sku = re.search(r"sku\s*:\s*'(.*)'", script.string).group(1)
			products.append(
				Product(
					name=name,
					price_min=price_min,
					price_max=price_max,
					inventory_status=inventory_status,
					page_crumbs=page_crumbs,
					sku=sku
				)
			)




		window.assetDomain = 'https://my.costco.com/assets/'; // Need to get from WCS property
	
None

				var isJQueryUpgraded = true;
			
None

			 function triggerStyleLoadEvent() {
			      	window.stylesLoaded = true;
			      	var event = document.createEvent('Event');
			      	event.initEvent("styles-loaded", true, true);
			      	document.dispatchEvent(event);
			 }
		
None

		var oneTrustLoadError = false;
		var optanonOnPageLoad=true;
		
		function displayOneTrustSystemDownErrorModal(){
			COSTCO.dialog.createExtDialogBox({
		        title: messages.ERR_ONETRUST_SYSTEM_DOWN_TITLE,
		        body: messages.ERR_ONETRUST_SYSTEM_DOWN_BODY,
		        primary: messages.CLOSE,
		        primary_event: function primary_event() {
		          COSTCO.dialog.close();
		        }
		      });
		}
		function verifyOTSystemDown(event){
			if(oneTrustLoadError){
				displayOneTrustSystemDownErrorModal();
				event.preventDefault();
			}
		}
	
None

	    		function OptanonWrapper(){
	    			$("#o

In [67]:
print(scripts[16].string)
pattern = r"window\.digitalData\s*=\s*\{[\s\S]*?\};?"
match = re.search(pattern, scripts[16].string)
print(match.group(0))




		            window.digitalData = {
		                siteLang :  'en',
		                siteReleaseEnv : 'prod',
		                siteStore : 'bc',
		                siteCountry : 'us',
		                pageName : 'Product Detail | 4000147442',
		                pageType : 'Product Detail',
		                pageCrumbs : ["Grocery & Household Essentials", "Coffee", "K-Cups, Coffee Pods & Capsules"],
		                product : {
		                  pid : '4000147442',
		                  sku : '4054232',
		                  prodDetailType : 'na',
		                  name : 'Kirkland Signature Coffee Organic Pacific Bold K-Cup Pod, 120-count',
		                  inventoryStatus : 'in stock',
		                  priceMin : '31.99',
		                  priceMax : '37.99',
		                  membershipReq : 'na',  
		                },
		            }
		            window.digitalDataEvents = [];
		    
window.digitalData = {
		                siteLang :  'en',
		                sit

In [102]:
import re
import json

"""window.digitalData = {
		                siteLang :  'en',
		                siteReleaseEnv : 'prod',
		                siteStore : 'bc',
		                siteCountry : 'us',
		                pageName : 'Product Detail | 4000147442',
		                pageType : 'Product Detail',
		                pageCrumbs : ["Grocery & Household Essentials", "Coffee", "K-Cups, Coffee Pods & Capsules"],
		                product : {
		                  pid : '4000147442',
		                  sku : '4054232',
		                  prodDetailType : 'na',
		                  name : 'Kirkland Signature Coffee Organic Pacific Bold K-Cup Pod, 120-count',
		                  inventoryStatus : 'in stock',
		                  priceMin : '31.99',
		                  priceMax : '37.99',
		                  membershipReq : 'na',  
		                }
"""

name = re.search(r"name\s*:\s*'(.*)'", scripts[16].string).group(1)
price_min = re.search(r"priceMin\s*:\s*'(.*)'", scripts[16].string).group(1)
price_max = re.search(r"priceMax\s*:\s*'(.*)'", scripts[16].string).group(1)
inventory_status = re.search(r"inventoryStatus\s*:\s*'(.*)'", scripts[16].string).group(1)
page_crumbs = re.search(r"pageCrumbs\s*:\s*\[(.*)\]", scripts[16].string).group(1)
sku = re.search(r"sku\s*:\s*'(.*)'", scripts[16].string).group(1)

print(name, price_min, price_max, inventory_status, page_crumbs, sku)


Kirkland Signature Coffee Organic Pacific Bold K-Cup Pod, 120-count 31.99 37.99 in stock "Grocery & Household Essentials", "Coffee", "K-Cups, Coffee Pods & Capsules" 4054232


JSONDecodeError: Expecting value: line 2 column 32 (char 33)

In [85]:
json_str

'{\n\t\t                "siteLang":  \'en\',\n\t\t                "siteReleaseEnv": \'prod\',\n\t\t                "siteStore": \'bc\',\n\t\t                "siteCountry": \'us\',\n\t\t                "pageName": \'Product Detail | 4000147442\',\n\t\t                "pageType": \'Product Detail\',\n\t\t                "pageCrumbs": ["Grocery & Household Essentials", "Coffee", "K-Cups, Coffee Pods & Capsules"],\n\t\t                "product": {\n\t\t                  "pid": \'4000147442\',\n\t\t                  "sku": \'4054232\',\n\t\t                  "prodDetailType": \'na\',\n\t\t                  "name": \'Kirkland Signature Coffee Organic Pacific Bold K-Cup Pod, 120-count\',\n\t\t                  "inventoryStatus": \'in stock\',\n\t\t                  "priceMin": \'31.99\',\n\t\t                  "priceMax": \'37.99\',\n\t\t                  "membershipReq": \'na\',  \n\t\t                }'

In [61]:
match = re.search(pattern, scripts[16].string.strip())
print(match)
# print(scripts[16].string)

None


In [54]:
print(scripts)

[<script type="text/javascript">
		window.assetDomain = 'https://my.costco.com/assets/'; // Need to get from WCS property
	</script>, <script>
				var isJQueryUpgraded = true;
			</script>, <script>
			 function triggerStyleLoadEvent() {
			      	window.stylesLoaded = true;
			      	var event = document.createEvent('Event');
			      	event.initEvent("styles-loaded", true, true);
			      	document.dispatchEvent(event);
			 }
		</script>, <script src="/wcsstore/builds/3.0.87076.0/RWDStaticAssets/js/built/mainjQueryBrowseUpgraded.min.js"></script>, <script defer="" src="/wcsstore/builds/3.0.87076.0/RWDStaticAssets/js/built/all.browse.min.js"></script>, <script defer="" src="/wcsstore/builds/3.0.87076.0/RWDStaticAssets/js/built/global.hbp.module.min.js" type="text/javascript"></script>, <script defer="" src="/wcsstore/builds/3.0.87076.0/RWDStaticAssets/js/built/product.module.min.js" type="text/javascript"></script>, <script defer="" src="/wcsstore/builds/3.0.87076.0/RWDStaticAssets/js

In [33]:
products

[]

In [4]:
import os
os.chdir("/home/carnufex/smart_shopper")

In [ ]:
products[0].dict()

In [ ]:
json.loads(match.group(0).split("= ")[1].replace(";", ""))

In [ ]:
import pickle
with open("costco.pickle", "wb") as f:
    pickle.dump(grocery, f)

In [ ]:
pickle.load(open("costco.pickle", "rb"))

In [1]:
%load_ext autoreload
%autoreload 2


In [5]:
from __future__ import annotations

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate

# from smart_shopper.llm import llm

from pydantic import BaseModel, Field

class Product(BaseModel):
    name: str  = Field(description="The name of the product")
    discount: str = Field(description="The discount on the product")


class GroupedProduct(Product):
    category: Catagories = Field(description="The category of the product")

    class Config:
        use_enum_values = True

PRODUCT_PROMPT = (
            """
            You are going to be given a product name and a discount. You will need to generate a sentence that describes the product and the discount.

            For example if the product is "Charmin Ultra Soft Bath Tissue, 2-Ply, 213 Sheets, 30 Rolls" and the discount is "Save $5.00" you would generate the following sentence:

            "Charmin Ultra Soft Bath Tissue, 2-Ply, 213 Sheets, 30 Rolls"
            "Save $5.00"
            "Toliet Paper"
            
            The following product is the one you will need to generate a sentence for:
            {input}

            {format_instructions}

            """
        )



# parser = PydanticOutputParser(pydantic_object=GroupedProduct)

# category_prompt = PromptTemplate(
#     template=PRODUCT_PROMPT,
#     input_variables=["input"],
#     partial_variables={"format_instructions": parser.get_format_instructions()},

# )
from __future__ import annotations

# from langchain_openai import ChatOpenAI

# from smart_shopper.settings import settings

# llm = ChatOpenAI(
#                 model="gpt-4o-mini",
#                 temperature=0,
#                 max_tokens=None,
#                 timeout=None,
#                 max_retries=2,
#                 api_key=settings.OPENAI_API_KEY,
#             )
# category_chain = category_prompt | llm | parser


In [ ]:

grouped_products = []
for product in grocery:
    response = await category_chain.ainvoke({"input": product})
    grouped_products.append(response)

In [ ]:
grouped_products

In [51]:
pickle.dump(grouped_products, open("grouped_products.pickle", "wb"))

In [ ]:
coffee = Catagories.COFFEE
coffee.value

In [6]:
import pickle
grouped_products = pickle.load(open("/home/carnufex/smart_shopper/notebooks/grouped_products.pickle", "rb"))

In [8]:
from smart_shopper.models import Catagories

In [9]:
coffee_values = [product for product in grouped_products if product.category == Catagories.COFFEE.value]

In [10]:
coffee_values = [product.model_dump_json() for product in grouped_products if product.category == Catagories.COFFEE.value]

In [ ]:
coffee_values

In [ ]:
import sendgrid
import os
from sendgrid.helpers.mail import *

sg = sendgrid.SendGridAPIClient(api_key=settings.SENDGRID_API_KEY)
from_email = Email("supersmartshopperhelper@gmail.com")
to_email = To("ryan.bieber@protonmail.com")
subject = "Deals from Costco"
content = Content("text/plain", f"""

{coffee_values}

""")
mail = Mail(from_email, to_email, subject, content)
response = sg.client.mail.send.post(request_body=mail.get())
print(response.status_code)
print(response.body)
print(response.headers)

In [ ]:
import json
to_email = To("kathleen.bieber@outlook.com")

['{"name":"Kirkland Signature Coffee Organic Pacific Bold K-Cup Pod, 120-count","discount":"$6 OFF\\n$6 manufacturer\'s savings is valid 12/26/24 through 1/20/25. While supplies last. Limit 5 per member. Terms & Conditions\\n","category":"coffee"}', '{"name":"Kirkland Signature Coffee Organic Breakfast Blend K-Cup Pod, 120-count","discount":"$6 OFF\\n$6 manufacturer\'s savings is valid 12/26/24 through 1/20/25. While supplies last. Limit 5 per member. Terms & Conditions\\n","category":"coffee"}', '{"name":"Kirkland Signature Coffee Organic House Decaf K-Cup Pod, 120-count","discount":"$6 OFF\\n$6 manufacturer\'s savings is valid 12/26/24 through 1/20/25. While supplies last. Limit 5 per member. Terms & Conditions\\n","category":"coffee"}', '{"name":"Kirkland Signature Coffee Organic Summit Roast K-Cup Pod, 120-count","discount":"$6 OFF\\n$6 manufacturer\'s savings is valid 12/26/24 through 1/20/25. While supplies last. Limit 5 per member. Terms & Conditions\\n","category":"coffee"}', '{"name":"Dunkin’ Donuts Original Blend, 45 oz","discount":"$6.40 OFF\\n$6.40 manufacturer\'s savings is valid 12/26/24 through 1/20/25. While supplies last. Limit 5 per member. Terms & Conditions\\n","category":"coffee"}', '{"name":"Starbucks Caffe Verona Whole Bean Coffee, Dark Roast, 2.5 lbs","discount":"$6 OFF\\n$6 manufacturer\'s savings is valid 12/26/24 through 1/20/25. While supplies last. Limit 6 per member. Terms & Conditions\\n","category":"coffee"}', '{"name":"Folgers Instant Coffee Classic Roast Coffee, 16 oz","discount":"$2.70 OFF\\n$2.70 manufacturer\'s savings is valid 12/26/24 through 1/20/25. While supplies last. Limit 5 per member. Terms & Conditions\\n","category":"coffee"}', '{"name":"TNI King Coffee 3-in-1 Instant Coffeemix Sticks, Medium, 0.56 oz, 120-count","discount":"$3 OFF\\n$3 manufacturer\'s savings is valid 12/26/24 through 1/20/25. While supplies last. Terms & Conditions\\n","category":"coffee"}']
table_html = """<table border="1">
<tr>
    <th>Name</th>
    <th>Discount</th>
    <th>Category</th>
</tr>
"""
for item_str in coffee_values:
    item = json.loads(item_str)
    table_html += f"<tr><td>{item['name']}</td><td>{item['discount']}</td><td>{item['category']}</td></tr>"
table_html += "</table>"

content = Content("text/html", table_html)
mail = Mail(from_email, to_email, subject, content)
response = sg.client.mail.send.post(request_body=mail.get())
print(response.status_code)
print(response.body)
print(response.headers)